In [1]:
import pandas as pd

In [2]:
import plotly.express as px

In [3]:
import plotly.io as pio

In [4]:
import plotly.graph_objects as go

In [5]:
pio.templates.default = "plotly_white"
# Available templates:
#         ['ggplot2', 'seaborn', 'simple_white', 'plotly',
#          'plotly_white', 'plotly_dark', 'presentation', 'xgridoff',
#          'ygridoff', 'gridon', 'none']

In [6]:
data = pd.read_csv("rfm_data.csv")

In [7]:
print(data.head())

   CustomerID PurchaseDate  TransactionAmount ProductInformation  OrderID  \
0        8814   2023-04-11             943.31          Product C   890075   
1        2188   2023-04-11             463.70          Product A   176819   
2        4608   2023-04-11              80.28          Product A   340062   
3        2559   2023-04-11             221.29          Product A   239145   
4        9482   2023-04-11             739.56          Product A   194545   

   Location  
0     Tokyo  
1    London  
2  New York  
3    London  
4     Paris  


In [8]:
from datetime import datetime

In [9]:
# Convert 'PurchaseDate' to datetime
data['PurchaseDate'] = pd.to_datetime(data['PurchaseDate'])

In [10]:
# Calculate Recency
data['Recency'] = (datetime.now().date() - data['PurchaseDate'].dt.date).dt.days

In [11]:
# Calculate Frequency
frequency_data = data.groupby('CustomerID')['OrderID'].count().reset_index()
frequency_data.rename(columns={'OrderID':'Frequency'}, inplace=True)
# inplace=True returns None inplace=False returns a copy of the object with the operation performed. 
data = data.merge(frequency_data, on='CustomerID', how='left')
# how{‘left’, ‘right’, ‘outer’, ‘inner’, ‘cross’}, default ‘inner’
# Type of merge to be performed.

# left: use only keys from left frame, similar to a SQL left outer join; preserve key order.

# right: use only keys from right frame, similar to a SQL right outer join; preserve key order.

# outer: use union of keys from both frames, similar to a SQL full outer join; sort keys lexicographically.

# inner: use intersection of keys from both frames, similar to a SQL inner join; preserve the order of the left keys.

# cross: creates the cartesian product from both frames, preserves the order of the left keys.

In [24]:
# Calculate Monetary Value
monetary_data = data.groupby('CustomerID')['TransactionAmount'].sum().reset_index()
monetary_data.rename(columns={'TransactionAmount':'MonetaryValue'}, inplace=True)
data = data.merge(monetary_data, on='CustomerID', how='left')

In [22]:
# data.drop('MonetaryValue_x', inplace=True, axis=1)
# data.drop('MonetaryValue_y', inplace=True, axis=1)

In [39]:
# Define scoring criteria for each RFM value
recency_scores = [5, 4, 3, 2, 1]  # Higher score for lower recency (more recent)
frequency_scores = [1, 2, 3, 4, 5]  # Higher score for higher frequency
monetary_scores = [1, 2, 3, 4, 5]  # Higher score for higher monetary value

# Calculate RFM scores
data['RecencyScore'] = pd.cut(data['Recency'], bins=5, labels=recency_scores)
data['FrequencyScore'] = pd.cut(data['Frequency'], bins=5, labels=frequency_scores)
data['MonetaryScore'] = pd.cut(data['MonetaryValue'], bins=5, labels=monetary_scores)

In [44]:
# Convert RFM scores to Int
data['RecencyScore'] = data['RecencyScore'].astype(int)
data['FrequencyScore'] = data['FrequencyScore'].astype(int)
data['MonetaryScore'] = data['MonetaryScore'].astype(int)

In [46]:
#RFM Value Segmentation

#Calculate RFM Score
data['RFM_Score'] = data['RecencyScore'] + data['FrequencyScore'] + data['MonetaryScore']

# Create RFM segments based on the RFM score
segment_labels = ['Low-Value', 'Mid-Value', 'High-Value']
data['Value Segment'] = pd.qcut(data['RFM_Score'], q=3, labels = segment_labels)

In [49]:
# cut command creates equispaced bins but frequency of samples is unequal in each bin
# qcut command creates unequal size bins but frequency of samples is equal in each bin.
# enter image description here

#     >>> x=np.array([24,  7,  2, 25, 22, 29])
#     >>> x
#     array([24,  7,  2, 25, 22, 29])

#     >>> pd.cut(x,3).value_counts() #Bins size has equal interval of 9
#     (2, 11.0]        2
#     (11.0, 20.0]     0
#     (20.0, 29.0]     4

#     >>> pd.qcut(x,3).value_counts() #Equal frequecy of 2 in each bins
#     (1.999, 17.0]     2
#     (17.0, 24.333]    2

In [51]:
#RFM Segmentation

segment_counts = data['Value Segment'].value_counts().reset_index()
segment_counts.columns = ['Value Segemnt', 'Count']

pastel_colors = px.colors.qualitative.Pastel

#Create Bar Chart

fig_segment_dist = px.bar(segment_counts, x='Value Segemnt', y='Count', color='Value Segemnt',
                          color_discrete_sequence=pastel_colors,
                         title="RFM Value Segmentation Distribution")

fig_segment_dist.update_layout(xaxis_title='RFM Value Segemnt', yaxis_title='Count', showlegend=True)
fig_segment_dist.show()

In [54]:
#RFM Customer Segment
data['RFM Customer Segments'] = ''

# Assign RFM segments based on the RFM score

data.loc[data['RFM_Score']>=9, 'RFM Customer Segments'] = 'Champions'
data.loc[(data['RFM_Score']>=6) & (data['RFM_Score']<9), 'RFM Customer Segments'] = 'Potential Loyalists'
data.loc[(data['RFM_Score']>=5) & (data['RFM_Score']<6), 'RFM Customer Segments'] = 'At Risk Customers'
data.loc[(data['RFM_Score']>=4) & (data['RFM_Score']<5), 'RFM Customer Segments'] = "Can't Lose"
data.loc[(data['RFM_Score']>=3) & (data['RFM_Score']<4), 'RFM Customer Segments'] = 'Lost'

data.head()

,CustomerID,PurchaseDate,TransactionAmount,ProductInformation,OrderID,Location,Recency,Frequency,MonetaryValue,RecencyScore,FrequencyScore,MonetaryScore,RFM_Score,Value Segment,RFM Customer Segments
0,8814,2023-04-11,943.31,Product C,890075,Tokyo,146,1,943.31,1,1,2,4,Low-Value,Can't Lose
1,2188,2023-04-11,463.70,Product A,176819,London,146,1,463.70,1,1,1,3,Low-Value,Lost
2,4608,2023-04-11,80.28,Product A,340062,New York,146,1,80.28,1,1,1,3,Low-Value,Lost
3,2559,2023-04-11,221.29,Product A,239145,London,146,1,221.29,1,1,1,3,Low-Value,Lost
4,9482,2023-04-11,739.56,Product A,194545,Paris,146,1,739.56,1,1,2,4,Low-Value,Can't Lose


In [61]:
segment_product_count = data.groupby(['Value Segment','RFM Customer Segments']).size().reset_index(name = 'Count')
segment_product_count = segment_product_count.sort_values('Count', ascending=False)
fig_treemap_segment_product = px.treemap(segment_product_count, path=['Value Segment','RFM Customer Segments'] 
                                         ,values='Count', color='Value Segment', color_discrete_sequence=px.colors.qualitative.Pastel,
                                         title='RFM Customer Segments by Value')
fig_treemap_segment_product.show()

/home/hashstudioz/anaconda3/lib/python3.9/site-packages/plotly/express/_core.py:1637: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/home/hashstudioz/anaconda3/lib/python3.9/site-packages/plotly/express/_core.py:1637: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.



In [68]:
# Filter the data to include only the customers in the Champions segment

champions_segment = data[data['RFM Customer Segments'] == "Champions"]
fig = go.Figure()
fig.add_trace(go.Box(y=champions_segment['RecencyScore'], name='Recency'))
fig.add_trace(go.Box(y=champions_segment['FrequencyScore'], name='Frequency'))
fig.add_trace(go.Box(y=champions_segment['MonetaryScore'], name='Monetary'))

fig.update_layout(title='Distribution of RFM Values within Champions Segment',
                  yaxis_title='RFM Value',
                  showlegend=True)
fig.show()

In [69]:
# Analyze the correlation of the recency, frequency, and monetary scores within the champions segment
# Methods of correlation summarize the relationship between two variables in a single number 
# called the correlation coefficient. The correlation coefficient is usually represented using the symbol r, 
# and it ranges from -1 to +1.

# A correlation coefficient quite close to 0, but either positive or negative, implies little or no relationship
# between the two variables. A correlation coefficient close to plus 1 means a positive relationship between the
# two variables, with increases in one of the variables being associated with increases in the other variable.

# A correlation coefficient close to -1 indicates a negative relationship between two variables, with an increase
# in one of the variables being associated with a decrease in the other variable. 
# Correlation is best visible in Heatmap

In [72]:
correlation_matrix = champions_segment[['RecencyScore','FrequencyScore','MonetaryScore']].corr()
fig_heatmap = go.Figure(data=go.Heatmap(
                   z=correlation_matrix.values,
                   x=correlation_matrix.columns,
                   y=correlation_matrix.columns,
                   colorscale='RdBu',
                   colorbar=dict(title='Correlation')))

fig_heatmap.update_layout(title='Correlation Matrix of RFM Values within Champions Segment')

fig_heatmap.show()

In [77]:
import plotly.colors
pastel_color = plotly.colors.qualitative.Pastel
segment_counts = data['RFM Customer Segments'].value_counts()

# Create a bar chart to compare segment counts
fig = go.Figure(data=[go.Bar(x=segment_counts.index, y=segment_counts.values,
                            marker=dict(color=pastel_colors))])

# Set the color of the Champions segment as a different color
champions_color = 'rgb(158, 202, 225)'
fig.update_traces(marker_color=[champions_color if segment == 'Champions' else pastel_colors[i]
                                for i, segment in enumerate(segment_counts.index)],
                  marker_line_color='rgb(8, 48, 107)',
                  marker_line_width=1, opacity=0.6)

# Update the layout
fig.update_layout(title='Comparison of RFM Segments',
                  xaxis_title='RFM Segments',
                  yaxis_title='Number of Customers',
                  showlegend=False)

fig.show()

In [84]:
# Calculate the average Recency, Frequency, and Monetary scores for each segment
segment_scores = data.groupby('RFM Customer Segments')['RecencyScore', 'FrequencyScore','MonetaryScore'].mean().reset_index()
segment_scores
fig = go.Figure()

# Add bars for Recency score
fig.add_trace(go.Bar(
    x=segment_scores['RFM Customer Segments'],
    y=segment_scores['RecencyScore'],
    name='Recency Score',
    marker_color='rgb(158,202,225)'
))

# Add bars for Frequency score
fig.add_trace(go.Bar(
    x=segment_scores['RFM Customer Segments'],
    y=segment_scores['FrequencyScore'],
    name='Frequency Score',
    marker_color='rgb(94,158,217)'
))

# Add bars for Monetary score
fig.add_trace(go.Bar(
    x=segment_scores['RFM Customer Segments'],
    y=segment_scores['MonetaryScore'],
    name='Monetary Score',
    marker_color='rgb(32,102,148)'
))

fig.update_layout(
    title='Comparison of RFM Segments based on Recency, Frequency, and Monetary Scores',
    xaxis_title='RFM Segments',
    yaxis_title='Score',
    barmode='group',
    showlegend=True
)

fig.show()
print(segment_scores)

/tmp/ipykernel_21246/2124192738.py:2: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.



  RFM Customer Segments  RecencyScore  FrequencyScore  MonetaryScore
0     At Risk Customers      2.344444        1.011111       1.644444
1            Can't Lose      1.537572        1.000000       1.462428
2             Champions      3.806452        3.064516       3.225806
3                  Lost      1.000000        1.000000       1.000000
4   Potential Loyalists      3.918489        1.194831       1.741551


# The End